In [1]:
import dicom
import os
import numpy as np
import glob
import cv2
%matplotlib inline
import matplotlib.pyplot as py
import SimpleITK as sitk
from PIL import Image
from scipy.misc import imrotate

In [2]:
def getAlignImg(t,label = None):#!!!notice, only take uint8 type for the imrotate function!!!
    f = lambda x:np.asarray([float(a) for a in x]);
    o = f(t.ImageOrientationPatient);
    o1 = o[:3];
    o2 = o[3:];
    oh = np.cross(o1,o2);
    or1 = np.asarray([0.6,0.6,-0.2]);
    o2new = np.cross(oh,or1);
    theta = np.arccos(np.dot(o2,o2new)/np.sqrt(np.sum(o2**2)*np.sum(o2new**2)))*180/3.1416;
    theta = theta * np.sign(np.dot(oh,np.cross(o2,o2new)));
    im_max = np.percentile(t.pixel_array.flatten(),99);
    res = imrotate(np.array(np.clip(np.array(t.pixel_array,dtype=np.float)/im_max*256,0,255),dtype=np.uint8),90);
    if label is None:
        return res;
    else:
        lab = imrotate(label,theta);
        return res,lab

In [3]:
def Orientation_M1 (img):
    if img.InPlanePhaseEncodingDirection=='COL':
        img_new=cv2.transpose(img.pixel_array)
        #py.imshow(img_new)
        img_new = cv2.flip(img_new, 0)
        return img_new
    else:
        #print 'Row Oriented'
        return img

In [4]:
def get_NewOrientation(img_array):
    y, x = np.nonzero(img_array)
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([x, y])
    cov = np.cov(coords)
    evals, evecs = np.linalg.eig(cov)
    sort_indices = np.argsort(evals)[::-1]
    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
    x_v2, y_v2 = evecs[:, sort_indices[1]]
    return (x_v1,y_v1), (x_v2,y_v2)

In [5]:
Kaggle_train = "/Users/laurawilke/Documents/DSE/lwilke/Capstone/Data/Sample data ( kaggle)/*/study/*/*"
#Get all the file names for the Kaggle Images
lstFilesKaggle_train=glob.glob(Kaggle_train)
#Get a list of all the paths for the sax images
lstFilesKaggle_train_sax = [s for s in lstFilesKaggle_train if "sax" in s]

In [8]:
img={}
get_align_img=[]
M1=[]

for i in range(37):
    rand=np.random.randint(0,len(lstFilesKaggle_train_sax))
    tmp_img=dicom.read_file(lstFilesKaggle_train_sax[rand])
    img[lstFilesKaggle_train_sax[rand]]=tmp_img
    align=getAlignImg(tmp_img,label=None)
    get_align_img.append(align)
    orient=Orientation_M1(tmp_img)
    M1.append(orient)

Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented
Row Oriented


In [9]:
keys=img.keys()

In [12]:
original=get_NewOrientation(img[keys[0]].pixel_array)
get_align_or=get_NewOrientation(get_align_img[0])
M1_or=get_NewOrientation(M1[0])
print original
print get_align_or
print M1_or

((0.99620435226664694, -0.08704532454411433), (0.08704532454411433, 0.99620435226664694))
((-0.373785746717963, -0.92751507564594049), (-0.92751507564594049, 0.373785746717963))
((-0.070182368805234874, -0.99753417741393013), (-0.99753417741393013, 0.070182368805234874))


In [13]:
original=get_NewOrientation(img[keys[2]].pixel_array)
get_align_or=get_NewOrientation(get_align_img[2])
M1_or=get_NewOrientation(M1[2])
print original
print get_align_or
print M1_or

((-0.059014867157064999, -0.99825710388378097), (-0.99825710388378097, 0.059014867157064999))
((-0.35813747532260026, -0.9336688646246879), (-0.9336688646246879, 0.35813747532260026))
((-0.087120166581441574, -0.99619780996287188), (-0.99619780996287188, 0.087120166581441574))


In [14]:
original=get_NewOrientation(img[keys[15]].pixel_array)
get_align_or=get_NewOrientation(get_align_img[15])
M1_or=get_NewOrientation(M1[15])
print original
print get_align_or
print M1_or

((0.99667802706424469, -0.081442681484125368), (0.081442681484125368, 0.99667802706424469))
((-0.36474711731689674, -0.93110662139682687), (-0.93110662139682687, 0.36474711731689674))
((-0.081341924680573913, -0.99668625519230469), (-0.99668625519230469, 0.081341924680573913))


In [15]:
original=get_NewOrientation(img[keys[30]].pixel_array)
get_align_or=get_NewOrientation(get_align_img[30])
M1_or=get_NewOrientation(M1[30])
print original
print get_align_or
print M1_or

((-0.20805728298674747, -0.9781166428377408), (-0.9781166428377408, 0.20805728298674747))
((-0.36362733432022448, -0.93154450335728345), (-0.93154450335728345, 0.36362733432022448))
((-0.081341924680573913, -0.99668625519230469), (-0.99668625519230469, 0.081341924680573913))
